In [9]:
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer

# Load the merged dataset
data = pd.read_csv('merged_data_updated.csv')

# Check for missing values
missing_values = data.isnull().sum()
print("Missing values in each column:\n", missing_values[missing_values > 0])

# Identify columns with all missing values
all_missing_columns = missing_values[missing_values == len(data)].index
print("Columns with all missing values:\n", all_missing_columns)

# Drop columns with all missing values
data_dropped = data.drop(columns=all_missing_columns)

# Separate numeric and non-numeric columns
numeric_cols = data_dropped.select_dtypes(include=['number']).columns
non_numeric_cols = data_dropped.select_dtypes(exclude=['number']).columns

# Handle missing values for numeric columns
imputer_numeric = SimpleImputer(strategy='mean')
data_numeric_imputed = pd.DataFrame(imputer_numeric.fit_transform(data_dropped[numeric_cols]), columns=numeric_cols)

# Handle missing values for non-numeric columns
imputer_non_numeric = SimpleImputer(strategy='most_frequent')
data_non_numeric_imputed = pd.DataFrame(imputer_non_numeric.fit_transform(data_dropped[non_numeric_cols]), columns=non_numeric_cols)

# Combine the imputed data
data_imputed = pd.concat([data_numeric_imputed, data_non_numeric_imputed], axis=1)

# Check if ProductCategory is present and print the columns
print("Columns after imputation:\n", data_imputed.columns)

# One-hot encode the 'ProductCategory' column if present
if 'ProductCategory' in data_imputed.columns:
    data_imputed = pd.get_dummies(data_imputed, columns=['ProductCategory'], drop_first=True)

# Feature selection and preprocessing
features = data_imputed.drop(columns=['Year', 'Month', 'Sales(In ThousandDollars)'], errors='ignore')
target = data_imputed['Sales(In ThousandDollars)']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Gradient Boosting Regressor
gbr = GradientBoostingRegressor()
gbr.fit(X_train, y_train)
gbr_predictions = gbr.predict(X_test)
gbr_mae = mean_absolute_error(y_test, gbr_predictions)
print(f'Gradient Boosting Regressor MAE: {gbr_mae}')

# Generate the date sequence for 36 years from the merged data
date_sequence = data_imputed[['Year', 'Month']].drop_duplicates()
date_sequence['Year'] = date_sequence['Year'].astype(int)
date_sequence['Month'] = date_sequence['Month'].astype(int)
date_sequence['Date'] = pd.to_datetime(date_sequence['Year'].astype(str) + '-' + date_sequence['Month'].astype(str).str.zfill(2))
date_sequence = date_sequence.sort_values('Date').reset_index(drop=True)
date_sequence['Year_Sequential'] = range(1, len(date_sequence) + 1)

# Prepare the Kaggle submission DataFrame
kaggle_submission = pd.DataFrame({'Year': range(1, 37)})
predictions = []

# Loop over each year in the Kaggle submission and make predictions
for seq_year in kaggle_submission['Year']:
    if seq_year in date_sequence['Year_Sequential'].values:
        date = pd.Timestamp(date_sequence[date_sequence['Year_Sequential'] == seq_year]['Date'].values[0])
        feature_row = data_imputed[(data_imputed['Year'] == date.year) & (data_imputed['Month'] == date.month)]

        # Make sure the feature_row is non-empty and has the same columns as the training data
        if not feature_row.empty:
            feature_row = feature_row.drop(columns=['Year', 'Month', 'Sales(In ThousandDollars)'])

            # Debugging: Print the feature row before prediction
            print(f'Feature row for Sequential Year {seq_year} ({date.strftime("%m/%Y")}):\n', feature_row)

            gbr_prediction = gbr.predict(feature_row)
            predictions.append(gbr_prediction.mean())  # Use the mean if there are multiple rows
        else:
            print(f'No data available for Sequential Year {seq_year}')
            predictions.append(0)  # Default to 0 if no data is available for the year
    else:
        print(f'Sequential Year {seq_year} not found in date sequence')
        predictions.append(0)  # Default to 0 if the sequential year is not found

# Fill in the Sales(In ThousandDollars) column with predictions
kaggle_submission['Sales(In ThousandDollars)'] = predictions

# Save the updated submission file
submission_path = 'submission_updated.csv'
kaggle_submission.to_csv(submission_path, index=False)

# Confirm the file is saved
print(f'Saved the updated submission file to {submission_path}')

Missing values in each column:
 Monthly Nominal GDP Index (inMillion$)         273
Monthly Real GDP Index (inMillion$)            273
CPI                                            273
unemployment rate                              273
CommercialBankInterestRateonCreditCardPlans    273
                                              ... 
Wind (km/h) low                                669
Wind (km/h) avg                                669
Wind (km/h) high                               669
Precip. (mm) sum                               669
WeatherEvent                                   669
Length: 63, dtype: int64
Columns with all missing values:
 Index(['Day', 'Temp high (°C)', 'Temp avg (°C)', 'Temp low (°C)',
       'Dew Point high (°C)', 'Dew Point avg (°C)', 'Dew Point low (°C)',
       'Humidity (%) high', 'Humidity (%) avg', 'Humidity (%) low',
       'Sea Level Press. (hPa) high', 'Sea Level Press. (hPa) avg',
       'Sea Level Press. (hPa) low', 'Visibility (km) high',
       'Vis